<img src="https://th.bing.com/th/id/R.0b9618782d4e7062573f5983d876649a?rik=7HWDbD%2feXNOe1A&pid=ImgRaw&r=0" width=150>


**I758 Wissens- und KI-basierte Systeme**

# Explorative Datenanalyse - Teil 1: Datenqualität erkunden & korrigieren
(c) Ricardo Knauer, Raphael Wallsberger, Christina Kratsch

In der Realität sind Datensätze selten so gutartig wie unsere Maschinendaten aus der letzten Übung. Regelmäßig kommt es im Alltag zu "unerwarteten" Datenwerten. Ein guter Data Scientist verwendet deshalb einen wesentlichen Anteil seiner Zeit damit, die Qualität der Daten zu überprüfen und ggf. für die gestellte Aufgabe optimieren. Dies ist eine Wissenschaft für sich (sogenanntes *Data Engineering*), hier erhalten Sie nur einen ersten Einblick. 

Nehmen wir an, wir hätten nur eine deutlich schlechtere Messreihe zur Verfügung. Alles, was wir wissen, ist, dass mit den Daten etwas "nicht in Ordnung" ist. Wir müssen uns langsam vortasten. Die schlechte Messreihe finden Sie in einer anderen Datei:

In [1]:
import pandas as pd

df = pd.read_csv("data/machine_data broken.csv", sep=";")
df.head()

,Maschine,Mode,Produkt,Strom / A,Drehmoment / Nm,Drehzahl / 1/min,Temp Umgebung / degC,Temp Umrichter / degC,Temp Werkzeug / degC,Bearbeitungszeit / s
0,A,2.0,X,NaN,"48,64",1463,"21,5","23,3","95,1","21,4"
1,B,2.0,Y,NaN,"50,92",1462,"20,9","22,8","98,2","24,8"
2,C,2.0,X,NaN,"46,85",1462,"21,3","23,8","93,1","21,4"
3,B,2.0,Y,NaN,"49,69",1463,"21,5","24,4","97,7",NaN
4,C,3.0,X,"26,195","53,29",1462,NaN,"25,3","104,1","19,9"


Der Datensatz ist deutlich kleiner - aber immerhin stimmt das Format:

In [ ]:
df.shape

Mit ```.info()``` bekommen Sie einen Einblick in die Datentypen jeder Spalte:

In [ ]:
df.info()

Hier zeigt sich das erste Problem: einige Werte scheinen in den Spalten zu fehlen, der ```Non-Null Count``` entspricht nämlich nicht für jede Spalte der Zeilenzahl.
Außerdem erkennen wir einen Fehler beim Einlesen der Daten: die meisten Spalten sind vom Typ ```object```. Soll heißen: Pandas erkennt nicht, dass es sich um numerische Werte handelt. Dies lässt sich einfach erklären - wir haben beim Einlesen nicht erwähnt, dass die Zahlen nicht mit der "deutschen" Schreibweise mit Komma als Trennzeichen notiert sind. Eine leidige und typische Fehlerquelle im Umgang mit Daten (die sich leider meist erst durch sehr kryptische Fehler weiter unten in den Pipeline bemerkbar macht).

Also nochmal - lassen Sie uns ```df``` nochmal überschreiben:

In [ ]:
df = pd.read_csv("data/machine_data broken.csv", sep=";", decimal=",")
df.info()

Viel besser! Aber da ist noch das Problem mit den fehlenden Werten. die letzte Spalte (Bearbeitungszeit) und auch einige andere Spalten scheinen unter Datenfehlern zu leiden.

Wir können es uns einfach machen. Nutzen Sie die  eingebauten Funktionen ```.isnull``` und ```.sum```, um die Nullwerte zu zählen:

Die Spalte *Bearbeitungszeit / s* scheint massiv von unserem Problem betroffen. Hier fehlen so viele Werte, dass wir davon ausgehen können, dass die Spalte praktisch wertlos ist. Entfernen Sie mit `del` die Spalte aus dem Data Frame!

Gut zu wissen: für komplizierte Verfahren gibt es die mächtige Funktion ```.dropna```, die Spalten anhand fester Qualitätskriterien entfernen kann. Die wollen wir hier aber heute **nicht** einsetzen. Führen Sie den auskommentierten Code NICHT aus.

In [ ]:
# Drop any rows which have any NaNs 
#df.dropna(axis=0) 
# Drop columns with over 70% non-NaNs 
#df.dropna(thresh=int(df.shape[0] * .7), axis=1)

Die Spalte *Strom / A* ist uns wichtig - aber auch hier fehlen massiv Werte. Schauen wir uns die Spalte nochmal an:

In [ ]:
df["Strom / A"]

Pandas bietet die Funktion ```.fillna```, mit der leere Werte gefunden und neu gesetzt werden können. Zum Beispiel auf 0:

In [ ]:
df['Strom / A'].fillna(0.0)

Achtung: achten Sie nochmal auf den Code der letzten Zeile, und dann schauen Sie sich nochmal die Werte in ```df``` an. Fällt Ihnen etwas auf? Wie hat sich ```df``` verändert?

In [ ]:
df

Noch mehr Aufgaben für Sie: wie sicher fühlen Sie sich mit Data Science? Versuchen Sie doch mal, die fehlenden Werte nicht durch 0.0, sondern durch etwas intelligenteres zu ersetzen, zum Beispiel den Mittelwert oder den Median. Schreiben sie die Spalte jetzt auch wieder in den Data Frame zurück, damit sich `df` tatsächlich ändert:

Plotten Sie nun zum Abschluss ihre Daten nochmal, zum Beispiel als Scatter Plot zwischen Strom und Drehmoment und betrachten Sie das Ergebnis Ihrer Arbeit. Vielleicht fällt Ihnen ja noch ein drittes Problem in unserem Datensatz auf? Schauen Sie mal in die "Ecken" des Plots!

# Explorative Datenanalyse - Teil 2: Daten skalieren und verarbeiten mit `scikit learn`
(c) Christina Kratsch

Viele Machine Learning Algorithmen machen Annahmen über ihre Daten, z.B. über die zugrundeliegende statistische Verteilung, den Wertebereich oder den Datentyp. Die Data Science Bibliothek `scikit learn` bietet neben einer extrem umfangreichen Sammlung an Algorithemn (und einer exzellenten Dokumentation) auch mit `sklearn.preprocessing` eine Bibliothek zur Vorverarbeitung von Daten.

## Skalieren und Normalisieren

Zuerst importieren wir die notwendigen Bibliotheken und laden unsere Daten.

In [3]:
import numpy as np
from sklearn import preprocessing

# Angenommen, dies sind unsere Daten
X_train = np.array([[1., -1., 2.],
                    [2., 0., 0.],
                    [0., 1., -1.]])

Die Standardisierung von Datensätzen ist eine gängige Anforderung für viele Machine-Learning-Schätzer. Sie könnten sich schlecht verhalten, wenn die einzelnen Merkmale nicht mehr oder weniger wie standardnormalverteilte Daten aussehen: Gaussian mit Null-Mittelwert und Einheitsvarianz.

**Aufgabe:** Überlegen Sie einen Moment, was ein Grund sein könnte, warum die Syntax des `StandardScaler` so umständlich ist. Schauen Sie ggf. auch in der [Dokumentation](https://scikit-learn.org/stable/modules/preprocessing.html) nach!

In [4]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
print(X_scaled)

[[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]]


**Aufgabe:**  legen Sie eine Kopie `df_num` Ihres Data Frames an. Nutzen Sie den StandardScaler, um alle Werte darin zu normalisieren. Funktioniert das? Was fällt Ihnen auf? Korrigieren Sie ggf. `df_num` entsprechend. Wie gehen Sie mit der Spalte `mode` um?

Manchmal möchte man Daten in einem bestimmten Wertebereich (z.B. [0, 1]) haben, z.B. um sie als Wahrscheinlichkeiten interpretieren zu können. Auch dafür bietet `sklearn.preprocessing` Möglichkeiten:

In [9]:
X_train = np.array([[1., -1., 2.],
                    [2., 0., 0.],
                    [0., 1., -1.]])

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

**Aufgabe:** Skalieren Sie die Einträge in `df_num` auf den Wertebereich [-3, 3]!

## Umgang mit kategorischen Variablen

Wir haben noch einige Spalten in unseren Maschinendaten, die wir nicht adressiert haben: die kategorischen Werte wie z.B. die Produkt-Klasse. Manche ML-Verfahren können aber grundsätzlich nur mit numerischen Werten umgehen. 

Die einfachste Möglichkeit, Kategorien in Zahlen umzuwandeln, ist, diese einfach zu "übersetzen", wie es im folgenden Beispiel mit drei Variablen passiert:

In [12]:
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from US', 'uses Safari']])

array([[0., 1., 1.]])

**Aufgabe:** erweitern Sie das Beispiel um weitere Belegungen (zum Beispiel das Geschlecht _non_binary_ oder die Herkunft _from Korea_) und kodieren Sie eine Beispiel-Belegung.

**Aufgabe:** was passiert, wenn einer der Werte nicht definiert ist (nutzen Sie `np.nan`)? Was macht die Belegung  `encoded_missing_value`?

array([[ 1.],
       [ 0.],
       [nan],
       [ 0.]])

**Aufgabe:** Zum Schluss wird es nochmal kniffelig. Informieren Sie sich, was ein [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) macht und kodieren Sie eine kategorische Spalte unseres Maschinendatensatzes (`df`). Achten Sie, wenn möglich, auch darauf, was mit fehlenden Werten passiert!

Klasse! Sie haben jetzt einen grundlegenden Überblick über die Möglichkeiten zur Manipulation von Daten gewonnen. Nicht alle Methoden erscheinen Ihnen vielleicht jetzt bereits umfassend sinnvoll - wenn wir weiter voranschreiten, werden Sie aber viele ML-Algorithmen kennenlernen, die auf den hier genannten Verfahren aufbauen. Wenn die Qualität Ihres ML-Tools einmal nicht gut ist, kehren Sie in dieses Tutorial zurück und überlegen Sie, wie Sie vielleicht die Ausgangsdaten "optimieren" können!